# 1. Creating a ROS package by hand.
http://wiki.ros.org/ROS/Tutorials/Creating%20a%20Package%20by%20Hand

## setup environment

In [17]:
import os
import sys 

home_dir = os.getcwd()
home_dir

'/home/pi/notebooks/Tutorials/1_Core_ROS_Tutorials/Intermediate_Level'

In [18]:
# get ROS related environment variables
envs = !printenv | grep ROS 
ROS_ENVs = {}

for env in envs:
    env = env.split('=')
    [k, v] = env
    ROS_ENVs[k] = v

ROS_ENVs['LD_LIBRARY_PATH'] = '/opt/ros/kinetic/lib'
ROS_ENVs['PKG_CONFIG_PATH'] = '/opt/ros/kinetic/lib/pkgconfig'
ROS_ENVs['CMAKE_PREFIX_PATH'] = '/opt/ros/kinetic'  
del ROS_ENVs['PWD']

In [19]:
# define workspace path
ws_dir = 'catkin_ws'
ws_dir_abs = os.path.join(home_dir, ws_dir)
src_dir_abs = os.path.join(ws_dir_abs, 'src') 

# append environment paths
def gen_paths(ws_dir_abs, src_dir_abs):
    paths = {}
    paths['ROS_PACKAGE_PATH'] = src_dir_abs
    paths['LD_LIBRARY_PATH'] = os.path.join(ws_dir_abs + 'devel/lib')
    paths['ROSLISP_PACKAGE_DIRECTORIES'] = os.path.join(ws_dir_abs + 'devel/share/common-lisp')
    paths['PKG_CONFIG_PATH'] = os.path.join(ws_dir_abs + 'devel/lib/pkgconfig')
    paths['CMAKE_PREFIX_PATH'] = os.path.join(ws_dir_abs + 'devel/lib')
    paths['LD_LIBRARY_PATH'] = os.path.join(ws_dir_abs + 'devel')
    return paths

def append_paths(ROS_ENVs, paths): 
    for path in ROS_ENVs.keys():
        os.environ[path] = ROS_ENVs[path]
        if path in paths.keys():
            os.environ[path] = '{}:{}'.format(paths[path], os.environ[path])

append_paths(ROS_ENVs, 
             gen_paths(ws_dir_abs, src_dir_abs))

## create package
There is a tool for creating ROS Packages (catkin_create_pkg), but, as you will see, there is nothing actually difficult here. catkin_create_pkg prevents mistakes and saves effort, but packages are just a directory and a simple XML file.

Now we'll create a new foobar package. This tutorial assumes that we're working your catkin workspace and sourcing of the setup file is already done.

In [20]:
rm -rf {ws_dir_abs}

In [21]:
mkdir -p {src_dir_abs}

In [22]:
cd {src_dir_abs}

/home/pi/notebooks/Tutorials/1_Core_ROS_Tutorials/Intermediate_Level/catkin_ws/src


In [23]:
package_name = 'foobar'

In [24]:
mkdir -p {package_name}

In [25]:
cd {package_name}

/home/pi/notebooks/Tutorials/1_Core_ROS_Tutorials/Intermediate_Level/catkin_ws/src/foobar


The very first thing we'll do is add our manifest file. The package.xml file allows tools like rospack to determine information about what your package depends upon.

Inside of foobar/package.xml put the following:

In [26]:
text = '''
<package>
  <name>foobar</name>
  <version>1.2.4</version>
  <description>
  This package provides foo capability.
  </description>
  <maintainer email="foobar@foo.bar.willowgarage.com">PR-foobar</maintainer>
  <license>BSD</license>

  <buildtool_depend>catkin</buildtool_depend>

  <build_depend>roscpp</build_depend>
  <build_depend>std_msgs</build_depend>

  <run_depend>roscpp</run_depend>
  <run_depend>std_msgs</run_depend>
</package>
'''

file_name = 'package.xml'

with open(file_name, 'w') as f:
    f.writelines(text)

See also this page from catkin tutorial for further information on catkin/package.xml.

Now that your package has a manifest, ROS can find it. Try executing the command:

In [27]:
!rospack find {package_name}

/home/pi/notebooks/Tutorials/1_Core_ROS_Tutorials/Intermediate_Level/catkin_ws/src/foobar


If ROS is set up correctly you should see something like: /home/user/ros/catkin_ws_top/src/foobar. This is how ROS finds packages behind the scenes.

Note that this package now also has dependencies on roscpp and std_msgs.

Such dependencies are used by catkin to configure packages in the right order.

Now we need the CMakeLists.txt file so that catkin_make, which uses CMake for its more powerful flexibility when building across multiple platforms, builds the package.

In foobar/CMakeLists.txt put:

In [28]:
text = '''
cmake_minimum_required(VERSION 2.8.3)
project({})
find_package(catkin REQUIRED roscpp std_msgs)
catkin_package()
'''.format(package_name)

file_name = 'CMakeLists.txt'

with open(file_name, 'w') as f:
    f.writelines(text)

That's all you need to start building a package in ROS using catkin. Of course, if you want it to actually start building something, you're going to need to learn a couple more CMake macros. See our CMakeLists.txt guide for more information. Also always go back to beginner level tutorial (CreatingPackage and so on) to customize your package.xml and CMakeLists.txt.

In [29]:
cd {ws_dir_abs}

/home/pi/notebooks/Tutorials/1_Core_ROS_Tutorials/Intermediate_Level/catkin_ws


In [30]:
!catkin_make

Base path: /home/pi/notebooks/Tutorials/1_Core_ROS_Tutorials/Intermediate_Level/catkin_ws
Source space: /home/pi/notebooks/Tutorials/1_Core_ROS_Tutorials/Intermediate_Level/catkin_ws/src
Build space: /home/pi/notebooks/Tutorials/1_Core_ROS_Tutorials/Intermediate_Level/catkin_ws/build
Devel space: /home/pi/notebooks/Tutorials/1_Core_ROS_Tutorials/Intermediate_Level/catkin_ws/devel
Install space: /home/pi/notebooks/Tutorials/1_Core_ROS_Tutorials/Intermediate_Level/catkin_ws/install
Creating symlink "/home/pi/notebooks/Tutorials/1_Core_ROS_Tutorials/Intermediate_Level/catkin_ws/src/CMakeLists.txt" pointing to "/opt/ros/kinetic/share/catkin/cmake/toplevel.cmake"
####
#### Running command: "cmake /home/pi/notebooks/Tutorials/1_Core_ROS_Tutorials/Intermediate_Level/catkin_ws/src -DCATKIN_DEVEL_PREFIX=/home/pi/notebooks/Tutorials/1_Core_ROS_Tutorials/Intermediate_Level/catkin_ws/devel -DCMAKE_INSTALL_PREFIX=/home/pi/notebooks/Tutorials/1_Core_ROS_Tutorials/Intermediate_Level/catkin_ws/install

In [31]:
# define package name and path 
package_dir_abs = !rospack find {package_name}
package_dir_abs = package_dir_abs[0]
package_dir_abs

'/home/pi/notebooks/Tutorials/1_Core_ROS_Tutorials/Intermediate_Level/catkin_ws/src/foobar'

In [32]:
cd {home_dir}

/home/pi/notebooks/Tutorials/1_Core_ROS_Tutorials/Intermediate_Level
